In [1]:
%pip install qiskit==1.4.1
%pip install qiskit-ibm-runtime==0.41.1
%pip install qiskit-nature==0.7.2
%pip install git+https://github.com/qiskit-community/qiskit-algorithms.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 42.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.7 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.8/366.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 66.6 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[no

In [2]:
%pip install pyscf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 19.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 77.4 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install pyberny

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Real CODE

In [32]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, dft
from pyscf.geomopt import berny_solver as geomopt

from qiskit_ibm_runtime import EstimatorV2
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms.minimum_eigensolvers import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, errors
from qiskit_aer.noise.errors import depolarizing_error, pauli_error, ReadoutError, thermal_relaxation_error

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [33]:
# Initial geometries
CO2_start = '''
C  0.000000  0.000000  0.000000
O  0.000000  0.000000  1.160000
O  0.000000  0.000000 -1.160000
'''

NH3_start = '''
N  0.000000  0.000000  0.000000
H  0.000000  0.935000  0.381000
H  0.809000 -0.467000  0.381000
H -0.809000 -0.467000  0.381000
'''

NH2COOH_start = '''
C  0.000000  0.000000  0.000000
O  1.200000  0.000000  0.000000
O -1.200000  0.000000  0.000000
N  0.000000  1.400000  0.000000
H  0.000000  2.100000  0.800000
H  0.000000  2.100000 -0.800000
H  0.000000 -0.600000  0.900000
'''

def run_pyscf_opt(xyz_string, name, method='b3lyp', basis='cc-pvtz', maxiter=50):
    print(f"Starting optimization for {name}...")
    atom_lines = [l for l in xyz_string.strip().splitlines() if l.strip()]
    atom_spec = '; '.join([' '.join(l.split()) for l in atom_lines])

    mol = gto.M(atom=atom_spec, basis=basis, unit='Angstrom', verbose=0)
    mf = dft.RKS(mol)
    mf.xc = method

    mol_opt = geomopt.optimize(mf, maxsteps=maxiter)

    coords = mol_opt.atom_coords()
    symbols = [mol_opt.atom_symbol(i) for i in range(mol_opt.natm)]
    xyz_lines = [f"{s} {c[0]:.8f} {c[1]:.8f} {c[2]:.8f}" for s, c in zip(symbols, coords)]
    xyz_out = '\n'.join(xyz_lines) + '\n'

    print(f"Optimization for {name} finished.")
    return xyz_out

# Create output directory
os.makedirs('xyz', exist_ok=True)

# Run optimizations
co2_xyz = run_pyscf_opt(CO2_start, 'CO2')
nh3_xyz = run_pyscf_opt(NH3_start, 'NH3')
nh2cooh_xyz = run_pyscf_opt(NH2COOH_start, 'NH2COOH')

# Save to XYZ files
with open('xyz/CO2_opt.xyz','w') as f: f.write(co2_xyz)
with open('xyz/NH3_opt.xyz','w') as f: f.write(nh3_xyz)
with open('xyz/NH2COOH_opt.xyz','w') as f: f.write(nh2cooh_xyz)

print("\nOptimized XYZ geometries saved to ./xyz/")

Starting optimization for CO2...
Optimization for CO2 finished.
Starting optimization for NH3...
Optimization for NH3 finished.
Starting optimization for NH2COOH...
Optimization for NH2COOH finished.

Optimized XYZ geometries saved to ./xyz/


Noisy Simulator

In [35]:
# Define noise model
noise_model = NoiseModel()

# Depolarizing errors
error_1q = depolarizing_error(0.01, 1)  # 1% for 1-qubit gates
error_2q = depolarizing_error(0.02, 2)  # 2% for 2-qubit gates
noise_model.add_all_qubit_quantum_error(error_1q, ["h", "x", "y", "z", "s", "t"])
noise_model.add_all_qubit_quantum_error(error_2q, ["cx", "cz"])

# Readout error
readout_error = errors.ReadoutError([[0.95, 0.05], [0.10, 0.90]])
noise_model.add_all_qubit_readout_error(readout_error)

# Thermal relaxation (example)
t1 = 50e3   # microseconds
t2 = 70e3
gate_time = 100
relax_error = thermal_relaxation_error(t1, t2, gate_time)
# 1-qubit thermal relaxation
relax_1q = thermal_relaxation_error(t1, t2, gate_time)
noise_model.add_all_qubit_quantum_error(relax_1q, ["x", "h"])

# 2-qubit thermal relaxation (example: use tensor product for simplicity)
relax_2q = thermal_relaxation_error(t1, t2, gate_time).expand(thermal_relaxation_error(t1, t2, gate_time))
noise_model.add_all_qubit_quantum_error(relax_2q, ["cx"])

# Create AerSimulator with noise
sim = AerSimulator(noise_model=noise_model, shots=1024)

In [36]:
# Convert XYZ to PySCF string
def xyz_to_pyscf_string(xyz_string):
    atom_lines = [l for l in xyz_string.strip().splitlines() if l.strip()]
    return '; '.join([' '.join(l.split()) for l in atom_lines])

# Generate N-C distance geometry
def make_geometry(n_c_distance, nh3_xyz=nh3_xyz, co2_xyz=co2_xyz):
    nh3_atoms = [line.split() for line in nh3_xyz.strip().splitlines()]
    co2_atoms = [line.split() for line in co2_xyz.strip().splitlines()]

    n_coord = np.array([float(x) for x in nh3_atoms[0][1:]])
    c_coord = np.array([float(x) for x in co2_atoms[0][1:]])
    shift = n_c_distance - np.linalg.norm(c_coord - n_coord)

    co2_shifted = []
    for atom in co2_atoms:
        coord = np.array([float(x) for x in atom[1:]])
        coord[0] += shift
        co2_shifted.append((atom[0], coord.tolist()))

    geometry = [(atom[0], [float(x) for x in atom[1:]]) for atom in nh3_atoms] + co2_shifted
    return "; ".join([f"{atom} {x} {y} {z}" for atom, (x, y, z) in geometry])


In [37]:
def compute_energy(geometry_str):
    # PySCF driver
    driver = PySCFDriver(atom=geometry_str, basis="sto-3g", charge=0, spin=0, unit=DistanceUnit.ANGSTROM)
    es_problem = driver.run()

    # Active space
    transformer = ActiveSpaceTransformer(num_electrons=6, num_spatial_orbitals=6)
    es_problem = transformer.transform(es_problem)

    # Mapper
    mapper = JordanWignerMapper()

    # Ansatz
    uccsd_ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper
        )
    )

    # Transpile ansatz for simulator
    uccsd_ansatz = transpile(uccsd_ansatz, sim, optimization_level=1)

    # Estimator and VQE
    estimator = EstimatorV2(sim)
    optimizer = SLSQP(maxiter=50)
    vqe = VQE(estimator, ansatz=uccsd_ansatz, optimizer=optimizer)

    # Solve
    solver = GroundStateEigensolver(mapper, vqe)
    result = solver.solve(es_problem)
    return result.total_energies[0].real


In [ ]:
# Distances to scan
distances = np.linspace(1.5, 4.0, 8)
energies = []

for d in distances:
    geom_str = make_geometry(d)
    print(f"Running for N-C distance = {d:.2f} Å ...")
    energy = compute_energy(geom_str)
    energies.append(energy)

# Plot
plt.plot(distances, energies, marker="o")
plt.xlabel("N–C Distance (Å)")
plt.ylabel("Energy (Hartree)")
plt.title("Reaction Pathway NH3 + CO2 → NH2COOH")
plt.show()


Running for N-C distance = 1.50 Å ...
